In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import base64
import datetime
import os

In [2]:
print("Pipeline started")

Pipeline started


In [3]:
db_params = {
    "url":os.getenv("DATASOURCE_URL").split("//")[1],
    "user": os.getenv("DATASOURCE_USERNAME"),
    "password": os.getenv("DATASOURCE_PASSWORD"),     
}
connection_string = "postgresql://{user}:{password}@{url}".format(**db_params)
engine = create_engine(connection_string)

In [4]:
users = pd.read_sql("SELECT * FROM users", engine)
users['access_token'] = None
users

,id,email,name,refresh_token,access_token


In [ ]:
if users.empty:
    print("No users found in the database.")
    exit()

In [5]:
refresh_token = users.loc[0,'refresh_token']
client_id = "6a6355fbeb044695930d74e002d91214"
client_secret = "2722c709f97248889ba35a4f33069ced"

auth_string = f"{client_id}:{client_secret}"
auth_bytes = auth_string.encode('utf-8')
auth_base64 = base64.b64encode(auth_bytes).decode('utf-8')

url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {auth_base64}"
}
for i, user in users.iterrows():
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        response_json = response.json()
        users.loc[0,'access_token'] = response_json['access_token']
    else:
        print(f"Error: {response.content}")

In [7]:
top_artists = pd.read_sql("SELECT * FROM user_top_artist", engine)
top_artists

,rank,user_id,artist_id,artist_name,month,year


In [8]:
url = "https://api.spotify.com/v1/me/top/artists?time_range=short_term"
date = datetime.datetime.now()
year = date.strftime("%Y")
month = date.strftime("%m")
for i, user in users.iterrows():
    headers = {
        "Authorization": f"Bearer {users.loc[0,'access_token']}"
    }
    response = requests.get(url, headers=headers)
    artists =response.json()['items']
    for rank, artist in enumerate(artists):
        record = {
            "rank": rank+1,
            "user_id": user['id'],
            "year": year,
            "month": month,
            "artist_id": artist['id'],
            "artist_name": artist['name'],
        }
        record_df = pd.DataFrame(record, index=[0])
        top_artists = pd.concat([top_artists,record_df])
top_artists = top_artists.drop_duplicates(['user_id', 'artist_id', 'year', 'month'])
top_artists

,rank,user_id,artist_id,artist_name,month,year
0,1,1,06HL4z0CvFAxyc27GXpf02,Taylor Swift,06,2025
0,2,1,4tuJ0bMpJh08umKkEXKUI5,Gracie Abrams,06,2025
0,3,1,0avMDS4HyoCEP6RqZJWpY2,Ethel Cain,06,2025
0,4,1,2FXC3k01G6Gw61bmprjgqS,Hozier,06,2025
0,5,1,2qxJFvFYMEDqd7ui6kSAcq,Zedd,06,2025
0,6,1,3YQKmKGau1PzlVlkL1iodx,Twenty One Pilots,06,2025


In [10]:
top_artists.to_sql('user_top_artist', engine, index=False, if_exists='append')

6

In [ ]:
print("Pipeline Finished")